In [1]:
# Setup detectron2 logger
import argparse
import os
import random
import shutil
import time
import warnings
import pickle
import numpy as np
import math
import sys
import copy
import re
import pandas as pd
import matplotlib.pyplot as plt
import json
import cv2
from itertools import compress
from shapely.geometry import Polygon

import torch
import torch.nn as nn
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor,DefaultTrainer,HookBase
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer,ColorMode,GenericMask
from detectron2.structures import BoxMode
from detectron2.evaluation import COCOEvaluator,inference_on_dataset
from detectron2.data import build_detection_test_loader,DatasetMapper,build_detection_train_loader,MetadataCatalog,DatasetCatalog
import detectron2.data.transforms as T
import detectron2.utils.comm as comm

import ray
import time

In [2]:
# setup directory
root_dir = r'/Users/lovely_shufan/'
data_dir = root_dir + r'Dropbox (Edison_Lab@UGA)/Projects/Bioinformatics_modeling/am_fungi_segmentation/data/AM_classify2/test/'
output_dir = root_dir + r'Dropbox (Edison_Lab@UGA)/AMF/AMF Imaging 2021/4_writing/Nature Scientific Report/'
model_dir = root_dir + r'Dropbox (Edison_Lab@UGA)/AMF/AMF Imaging 2021/2_computer_vision/'

## Create a detectron2 config and a detectron2 DefaultPredictor to run inference 

In [3]:
classes=['root','AMF internal hypha','AMF external hypha','AMF arbuscule','AMF vesicle','AMF spore','others']


In [4]:
cfg = get_cfg() # return default configuration
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")) # copy config files from open source projects

# training configuration
cfg.DATASETS.TEST=()
cfg.DATALOADER.NUM_WORKERS=2
#cfg.SOLVER.IMS_PER_BATCH=args.batch_size

cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE=128 #Number of regions per image used to train RPN. faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES=len(classes)# (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)
cfg.MODEL.BACKBONE.FREEZE_AT=2
cfg.SEED=1
cfg.AUG_FLAG=1

# inference configuration
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7  # set threshold for this model
cfg.MODEL.WEIGHTS=os.path.join(model_dir, "Trainset1_model_best.pth") # path to the best model trained
cfg.MODEL.DEVICE='cpu' # use cpu for inference



In [5]:
# Define colors for each class in RGB format
my_thing_colors = [
    (255, 0, 0),       # Red for 'root'
    (0, 255, 0),       # Green for 'AMF internal hypha'
    (0, 0, 255),       # Blue for 'AMF external hypha'
    (255, 255, 0),     # Yellow for 'AMF arbuscule'
    (255, 0, 255),     # Magenta for 'AMF vesicle'
    (0, 255, 255),     # Cyan for 'AMF spore'
    (128, 128, 128)    # Gray for 'others'
]


my_thing_classes = ['root','AMF internal hypha','AMF external hypha','AMF arbuscule','AMF vesicle','AMF spore','others']

inf_metadata = MetadataCatalog.get('inference').set(thing_classes=my_thing_classes, thing_colors=my_thing_colors)



## do inference on an example image

In [15]:
testfilename = 'Snap-3946-Image Export-847.jpg'

In [6]:
testimg = cv2.imread(os.path.join(data_dir, 
                                  'Snap-3946-Image Export-847.jpg'),cv2.IMREAD_COLOR)

In [7]:
cv2.imshow('',testimg)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

In [8]:
predictor = DefaultPredictor(cfg)
outputs = predictor(testimg)

[02/17 14:43:49 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /Users/lovely_shufan/Dropbox (Edison_Lab@UGA)/AMF/AMF Imaging 2021/2_computer_vision/Trainset1_model_best.pth ...


The checkpoint state_dict contains keys that are not used by the model:
  pixel_mean
  pixel_std


In [9]:
v.metadata.get("thing_colors")

NameError: name 'v' is not defined

In [10]:
MetadataCatalog.get('inference')

namespace(name='inference',
          thing_classes=['root',
                         'AMF internal hypha',
                         'AMF external hypha',
                         'AMF arbuscule',
                         'AMF vesicle',
                         'AMF spore',
                         'others'],
          thing_colors=[(255, 0, 0),
                        (0, 255, 0),
                        (0, 0, 255),
                        (255, 255, 0),
                        (255, 0, 255),
                        (0, 255, 255),
                        (128, 128, 128)])

In [ ]:
for data in 

In [35]:
class CustomVisualizer(Visualizer):
    def _default_font_size(self):
        # Increase font size
        return super()._default_font_size() * 20

In [36]:
v = CustomVisualizer(testimg[:, :, ::-1], MetadataCatalog.get("inference"), scale=0.2)
v.overlay_instances(masks=outputs["instances"].pred_masks, alpha=0.1)  # Set mask transparency to 90%
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
out_img = out.get_image()[:, :, ::-1]  # Convert image back to BGR

In [13]:

cv2.imshow('',out_img)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)



-1

In [37]:
cv2.imwrite(os.path.join(output_dir, "_".join([testfilename.replace('.jpg', ''),'inf_result0.2-1.png'])), out_img)

True

In [47]:
testfilename2 = 'AH2O2(1h)-KOH(Overnight)-AH2O2(1h) (48)-Image Export-285.jpg'
testimg2 = cv2.imread(os.path.join(data_dir, testfilename2),cv2.IMREAD_COLOR)
predictor = DefaultPredictor(cfg)
output = predictor(testimg2)
v = CustomVisualizer(testimg2[:, :, ::-1], MetadataCatalog.get("inference"), scale=0.2)
out = v.draw_instance_predictions(output["instances"].to("cpu"))
out_img = out.get_image()[:, :, ::-1]
cv2.imwrite(os.path.join(output_dir, "_".join([testfilename2.replace('.jpg', ''),'inf_result-5.png'])), out_img)

[02/17 15:23:24 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /Users/lovely_shufan/Dropbox (Edison_Lab@UGA)/AMF/AMF Imaging 2021/2_computer_vision/Trainset1_model_best.pth ...


The checkpoint state_dict contains keys that are not used by the model:
  pixel_mean
  pixel_std


True

In [52]:
testfilename3 = 'Snap-720.jpg'
testimg3 = cv2.imread(os.path.join(data_dir, testfilename3),cv2.IMREAD_COLOR)
predictor = DefaultPredictor(cfg)
output = predictor(testimg3)
v = CustomVisualizer(testimg3[:, :, ::-1], MetadataCatalog.get("inference"), scale=0.2)
out = v.draw_instance_predictions(output["instances"].to("cpu"))
out_img = out.get_image()[:, :, ::-1]
cv2.imwrite(os.path.join(output_dir, "_".join([testfilename3.replace('.jpg', ''),'inf_result-1.png'])), out_img)

[02/17 15:32:04 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /Users/lovely_shufan/Dropbox (Edison_Lab@UGA)/AMF/AMF Imaging 2021/2_computer_vision/Trainset1_model_best.pth ...


The checkpoint state_dict contains keys that are not used by the model:
  pixel_mean
  pixel_std


True

In [53]:
testfilename4 = 'Snap-1886-Image Export-234.jpg'
testimg4 = cv2.imread(os.path.join(data_dir, testfilename4),cv2.IMREAD_COLOR)
predictor = DefaultPredictor(cfg)
output = predictor(testimg4)
v = CustomVisualizer(testimg4[:, :, ::-1], MetadataCatalog.get("inference"), scale=0.2)
out = v.draw_instance_predictions(output["instances"].to("cpu"))
out_img = out.get_image()[:, :, ::-1]
cv2.imwrite(os.path.join(output_dir, "_".join([testfilename4.replace('.jpg', ''),'inf_result.png'])), out_img)


[02/17 15:34:50 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /Users/lovely_shufan/Dropbox (Edison_Lab@UGA)/AMF/AMF Imaging 2021/2_computer_vision/Trainset1_model_best.pth ...


The checkpoint state_dict contains keys that are not used by the model:
  pixel_mean
  pixel_std


True

In [54]:
testfilename5 = 'Snap-5510-Image Export-260.jpg'
testimg5 = cv2.imread(os.path.join(data_dir, testfilename5),cv2.IMREAD_COLOR)
predictor = DefaultPredictor(cfg)
output = predictor(testimg5)
v = CustomVisualizer(testimg5[:, :, ::-1], MetadataCatalog.get("inference"), scale=0.2)
out = v.draw_instance_predictions(output["instances"].to("cpu"))
out_img = out.get_image()[:, :, ::-1]
cv2.imwrite(os.path.join(output_dir, "_".join([testfilename5.replace('.jpg', ''),'inf_result.png'])), out_img)


[02/17 15:38:05 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /Users/lovely_shufan/Dropbox (Edison_Lab@UGA)/AMF/AMF Imaging 2021/2_computer_vision/Trainset1_model_best.pth ...


The checkpoint state_dict contains keys that are not used by the model:
  pixel_mean
  pixel_std


True

In [55]:
testfilename6 = 'Snap-6212-Image Export-04.jpg'
testimg6 = cv2.imread(os.path.join(data_dir, testfilename6),cv2.IMREAD_COLOR)
predictor = DefaultPredictor(cfg)
output = predictor(testimg6)
v = CustomVisualizer(testimg6[:, :, ::-1], MetadataCatalog.get("inference"), scale=0.2)
out = v.draw_instance_predictions(output["instances"].to("cpu"))
out_img = out.get_image()[:, :, ::-1]
cv2.imwrite(os.path.join(output_dir, "_".join([testfilename6.replace('.jpg', ''),'inf_result.png'])), out_img)


[02/17 15:40:27 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /Users/lovely_shufan/Dropbox (Edison_Lab@UGA)/AMF/AMF Imaging 2021/2_computer_vision/Trainset1_model_best.pth ...


The checkpoint state_dict contains keys that are not used by the model:
  pixel_mean
  pixel_std


True

In [59]:
testfilename7 = 'Snap-6425-Image Export-217.jpg'
testimg7 = cv2.imread(os.path.join(data_dir, testfilename7),cv2.IMREAD_COLOR)
predictor = DefaultPredictor(cfg)
output = predictor(testimg7)
v = CustomVisualizer(testimg7[:, :, ::-1], MetadataCatalog.get("inference"), scale=0.2)
out = v.draw_instance_predictions(output["instances"].to("cpu"))
out_img = out.get_image()[:, :, ::-1]
cv2.imwrite(os.path.join(output_dir, "_".join([testfilename7.replace('.jpg', ''),'inf_result-2.png'])), out_img)


[02/17 15:43:03 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /Users/lovely_shufan/Dropbox (Edison_Lab@UGA)/AMF/AMF Imaging 2021/2_computer_vision/Trainset1_model_best.pth ...


The checkpoint state_dict contains keys that are not used by the model:
  pixel_mean
  pixel_std


True

In [65]:
testfilename8 = 'Snap-3642-Image Export-595.jpg'
testimg8 = cv2.imread(os.path.join(data_dir, testfilename8),cv2.IMREAD_COLOR)
predictor = DefaultPredictor(cfg)
output = predictor(testimg8)
v = CustomVisualizer(testimg8[:, :, ::-1], MetadataCatalog.get("inference"), scale=0.2)
out = v.draw_instance_predictions(output["instances"].to("cpu"))
out_img = out.get_image()[:, :, ::-1]
cv2.imwrite(os.path.join(output_dir, "_".join([testfilename8.replace('.jpg', ''),'inf_result-5.png'])), out_img)


[02/17 15:47:31 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /Users/lovely_shufan/Dropbox (Edison_Lab@UGA)/AMF/AMF Imaging 2021/2_computer_vision/Trainset1_model_best.pth ...


The checkpoint state_dict contains keys that are not used by the model:
  pixel_mean
  pixel_std


True

## Visualize groundtruth

In [91]:
def get_amseg_dicts(img_dir,classes):
    testfilenames = ['Snap-720.jpg','Snap-1886-Image Export-234.jpg','Snap-5510-Image Export-260.jpg','Snap-6425-Image Export-217.jpg']
    anno_file=os.path.join(img_dir,"regiondata.csv")
    annotab=pd.read_csv(anno_file,delimiter="\t")
    files=annotab['filename'].unique()
    dataset_dicts=[]
    for idx,file in enumerate(files):
        if file in testfilenames:
            record={}
            filename=os.path.join(img_dir,file)
            height,width=cv2.imread(filename).shape[:2]
            record["file_name"]=filename
            record["image_id"]=idx
            record["height"]=height
            record["width"]=width
            subtab=annotab[annotab['filename']==file]
            objs=[]
            for anno_i in range(subtab.shape[0]):#multiple masks/boxes
                tab_rec=subtab.iloc[anno_i]
                # assert not tab_rec["region_attributes"]#check it is []
                anno=json.loads(tab_rec["region_shape_attributes"])
                if len(anno)==0:
                    continue

                px=anno["all_points_x"]
                py=anno["all_points_y"]
                poly=[(x+0.5,y+0.5) for x,y in zip(px,py)]
                poly=[p for x in poly for p in x]
                category_id=np.where([ele==tab_rec['region_attributes'] for ele in classes])[0][0]
                obj={
                    "bbox":[np.min(px),np.min(py),np.max(px),np.max(py)],
                    "bbox_mode":BoxMode.XYXY_ABS,
                    "segmentation":[poly],
                    "category_id":category_id,
                }
                objs.append(obj)

            record["annotations"]=objs
            dataset_dicts.append(record)
    
    return dataset_dicts

In [66]:
testfilenames = ['Snap-720.jpg','Snap-1886-Image Export-234.jpg','Snap-5510-Image Export-260.jpg','Snap-6425-Image Export-217.jpg']

In [76]:
testfilepaths = []
for i in testfilenames:
    testfilepaths.append(data_dir+i)

In [68]:
classes=['root','AMF internal hypha','AMF external hypha','AMF arbuscule','AMF vesicle','AMF spore','others']

In [92]:
dataset_dicts=get_amseg_dicts(data_dir,classes)

In [93]:
len(dataset_dicts)

4

In [96]:
for d in dataset_dicts:
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
    v = CustomVisualizer(im[:, :, ::-1], 
                         MetadataCatalog.get("inference"), 
                         scale=0.2, 
                         instance_mode=ColorMode.IMAGE_BW)    
    out=v.draw_dataset_dict(d)
    cv2.imwrite(d["file_name"].replace('.jpg', '')+'_groundtruth.png',out.get_image()[:, :, ::-1])
    
    
    